### H2O

https://www.cienciadedatos.net/documentos/py04_machine_learning_con_h2o_y_python

In [1]:
!pip3 install h2o

You should consider upgrading via the 'z:\data_science\software\python\python_3.6.4\python.exe -m pip install --upgrade pip' command.


In [1]:
import pandas as pd
import numpy as np

from sklearn.metrics import mean_squared_error

import h2o
from h2o.automl import H2OAutoML

In [2]:
train = pd.read_csv('diamonds_train.csv')
test = pd.read_csv('diamonds_test.csv')

In [3]:
# hay que instalar JAVA versión < 14
# https://www.oracle.com/java/technologies/javase-jdk11-downloads.html

In [4]:
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...
; Java HotSpot(TM) Client VM (build 25.231-b11, mixed mode)
Z:\Data_Science\Software\Python\Python_3.6.4\lib\site-packages\h2o\backend\server.py:385: UserWarning:   You have a 32-bit version of Java. H2O works best with 64-bit Java.
  Please download the latest 64-bit Java SE JDK from Oracle.

  warn("  You have a 32-bit version of Java. H2O works best with 64-bit Java.\n"
  Starting server from Z:\Data_Science\Software\Python\Python_3.6.4\lib\site-packages\h2o\backend\bin\h2o.jar
  Ice root: C:\Users\gvazquez\AppData\Local\Temp\tmp305w5mfn
  JVM stdout: C:\Users\gvazquez\AppData\Local\Temp\tmp305w5mfn\h2o_gvazquez_started_from_python.out
  JVM stderr: C:\Users\gvazquez\AppData\Local\Temp\tmp305w5mfn\h2o_gvazquez_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,02 secs
H2O_cluster_timezone:,Europe/Paris
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.32.0.4
H2O_cluster_version_age:,4 months and 19 days !!!
H2O_cluster_name:,H2O_from_python_gvazquez_uzdbvk
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,247.5 Mb
H2O_cluster_total_cores:,0
H2O_cluster_allowed_cores:,0
H2O_cluster_status:,"accepting new members, healthy"


In [5]:
h2train = h2o.H2OFrame(train)
h2test = h2o.H2OFrame(test)

Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%


In [6]:
columnas = [a for a in h2train.columns if a != "price"][1:]
x = columnas
y = "price"
print(columnas)

['carat', 'cut', 'color', 'clarity', 'depth', 'table', 'x', 'y', 'z']


In [7]:
automl = H2OAutoML(max_models=50, seed=42, max_runtime_secs=300, sort_metric='RMSE')
automl.train(x=x, y=y, training_frame=h2train)
# en Windows no está disponible XGBoost :'(

AutoML progress: |
21:26:07.766: AutoML: XGBoost is not available; skipping it.

████████████████████████████████████████████████████████| 100%


In [8]:
print('[INFO] Models leader board:')
leader_board = automl.leaderboard
leader_board.head()

[INFO] Models leader board:


model_id,rmse,mean_residual_deviance,mse,mae,rmsle
StackedEnsemble_AllModels_AutoML_20210620_212607,524.572,275176,275176,265.168,0.0938705
StackedEnsemble_BestOfFamily_AutoML_20210620_212607,531.021,281984,281984,269.146,0.0967715
GBM_grid__1_AutoML_20210620_212607_model_10,532.143,283176,283176,272.761,0.099033
GBM_4_AutoML_20210620_212607,532.776,283850,283850,267.663,0.0924966
GBM_3_AutoML_20210620_212607,533.556,284682,284682,272.097,0.0980888
GBM_grid__1_AutoML_20210620_212607_model_5,534.292,285468,285468,272.373,nan
GBM_2_AutoML_20210620_212607,534.413,285598,285598,275.044,nan
GBM_grid__1_AutoML_20210620_212607_model_2,537.035,288407,288407,270.383,0.0934097
GBM_grid__1_AutoML_20210620_212607_model_9,538.6,290089,290089,273.964,nan
GBM_grid__1_AutoML_20210620_212607_model_7,542.448,294250,294250,278.287,nan


In [9]:
predictions = automl.leader.predict(h2test)

stackedensemble prediction progress: |████████████████████████████████████| 100%


In [10]:
test.rename(columns={"Unnamed: 0" : "id"}, inplace = True)

In [11]:
test["price"] = predictions.as_data_frame()
columnasentrega = ["id","price"]
entrega = test["price"]
test = test[columnasentrega]

In [12]:
test.head()

,id,price
0,0,414.954507
1,1,1699.129208
2,2,9550.341099
3,3,535.114304
4,4,9832.887516


In [13]:
test['price'].astype("int")

0          414
1         1699
2         9550
3          535
4         9832
         ...  
13444     4808
13445      457
13446    14941
13447    10794
13448      739
Name: price, Length: 13449, dtype: int32

In [14]:
test.to_csv('to_submit_int.csv', index=False)

In [15]:
testing = pd.read_csv('to_submit_int.csv')
testing

,id,price
0,0,414.954507
1,1,1699.129208
2,2,9550.341099
3,3,535.114304
4,4,9832.887516
...,...,...
13444,13444,4808.840639
13445,13445,457.755320
13446,13446,14941.798953
13447,13447,10794.781777


In [16]:
h2o.cluster().shutdown()

H2O session _sid_84bf closed.


**Evaluación con los datos reales**

In [17]:
true = pd.read_csv('evaluation_price.csv')

y_pred = testing['price']
y_true = true['price']
np.sqrt(mean_squared_error(y_true, y_pred))

526.3573341009826